In [15]:
import pandas as pd
import numpy as np

In [16]:
def data_cleaning():
    ## loading data
    # 航班資訊
    flight = pd.read_csv('data/flight_info_final.csv', dtype={"Flight Number": int})
    # rename
    flight = flight.rename(columns={'Carrier Code':'airline_code',
                                    'Date (MM/DD/YYYY)':'depa_date',
                                    'Flight Number':'flight_num',
                                    'Tail Number':'tail_num',
                                    'Destination Airport':'dest_airport',
                                    'Departure airport':'depa_airport',
                                    'Scheduled departure time':'sche_arriv_time',
                                    'Actual departure time':'actual_arriv_time',
                                    'Scheduled elapsed time (Minutes)':'sche_duration',
                                    'Actual elapsed time (Minutes)':'actu_duration',
                                    'Departure delay (Minutes)':'delay_depa_time',
                                    'Wheels-off time':'Wheels-off time',
                                    'Taxi-Out time (Minutes)':'Taxi-Out time (Minutes)',
                                    'Delay Carrier (Minutes)':'delay_carrier_time',
                                    'Delay Weather (Minutes)':'delay_weather_time',
                                    'Delay National Aviation System (Minutes)':'delay_NAS_time',
                                    'Delay Security (Minutes)':'delay_security_time',
                                    'Delay Late Aircraft Arrival (Minutes)':'delay_late_arrival_time'
                                    })
    # 機場資訊
    airport = pd.read_csv("data/airport_info_0325.csv")
    # 補滿5位數
    airport["WBAN"] = airport["WBAN"].astype(str).str.zfill(5)
    # 移除airport_code重複項
    airport.drop_duplicates(subset=['airport_code'], inplace=True)
    # 不選擇airport_name
    airport = airport[["state_code", "WBAN", "timezone", "airport_code", "hub_type"]]
    # Tail Number不是NA才要，沒有Tail Number代表沒飛
    flight = flight[flight['tail_num'].notna()]

    # float轉str
    flight['flight_num'] = flight['flight_num'].astype(str)

    # 刪除不必要欄位(wheels-off time & Taix-Out time (Minutes))
    flight = flight.drop(['Wheels-off time', 'Taxi-Out time (Minutes)'], axis= 1)

    # 將 Tail Number 開頭N刪除
    flight["tail_num"] = flight["tail_num"].str.lstrip("N")

    # 將 sche_arriv_time & actual_arriv_time 等於24:00替換成00:00
    flight["sche_arriv_time"] = flight["sche_arriv_time"].str.replace("24:00", "00:00")
    flight["actual_arriv_time"] = flight["actual_arriv_time"].str.replace("24:00", "00:00")

    # merge，為了時區轉換。
    # 出發機場時區
    flight_final = flight.merge(airport, left_on="depa_airport", right_on="airport_code", how = 'left')
    # 降落機場時區
    flight_final = flight_final.merge(airport, left_on = "dest_airport", right_on = "airport_code", how = 'left', suffixes=("_depa", "_dest"))



In [17]:
def data_time_clearning():
    # 將depa_date轉成datetime格式
    flight_final['depa_date'] = pd.to_datetime(flight_final['depa_date'])
    # 將depa_date跟sche_arriv_time結合 = sche_depa(預計出發時間)
    sche_depa = pd.to_datetime(flight_final['depa_date'].astype(str) + ' ' + flight_final['sche_arriv_time'].astype(str))
    # sche_depa + delay_depa_time(延誤出發) = actu_depa(實際出發時間)
    actu_depa = sche_depa + pd.to_timedelta(flight_final['delay_depa_time'], unit="m")
    # 預計抵達時間 = 預計出發時間 + 預計飛行時間
    sche_dest = sche_depa + pd.to_timedelta(flight_final['sche_duration'], unit="m")
    # 時區轉換
    sche_dest += pd.to_timedelta(-(flight_final["timezone_depa"].astype('float')) + flight_final["timezone_dest"].astype('float'), unit="h")
    # 實際抵達時間 = 實際出發時間 + 實際飛行時間
    actu_dest = actu_depa + pd.to_timedelta(flight_final['actu_duration'], unit="m")
    # 時區轉換
    actu_dest += pd.to_timedelta(-(flight_final["timezone_depa"].astype('float')) + flight_final["timezone_dest"].astype('float'), unit="h")

    # 延誤時長
    delay = actu_dest - sche_dest
    flight_final["delay_dest"] = delay.dt.total_seconds() / 60

    # 拆分預計出發年月日時分
    flight_final['sche_depa_year'] = sche_depa.dt.year
    flight_final['sche_depa_month'] = sche_depa.dt.month
    flight_final['sche_depa_day'] = sche_depa.dt.day
    flight_final['sche_depa_hr'] = sche_depa.dt.hour
    flight_final['sche_depa_min'] = sche_depa.dt.minute
    # 拆分實際出發年月日時分
    flight_final['actu_depa_year'] = actu_depa.dt.year
    flight_final['actu_depa_month'] = actu_depa.dt.month
    flight_final['actu_depa_day'] = actu_depa.dt.day
    flight_final['actu_depa_hr'] = actu_depa.dt.hour
    flight_final['actu_depa_min'] = actu_depa.dt.minute
    # 拆分預計抵達年月日時分
    flight_final['sche_dest_year'] = sche_dest.dt.year
    flight_final['sche_dest_month'] = sche_dest.dt.month
    flight_final['sche_dest_day'] = sche_dest.dt.day
    flight_final['sche_dest_hr'] = sche_dest.dt.hour
    flight_final['sche_dest_min'] = sche_dest.dt.minute
    # 拆分實際抵達年月日時分
    flight_final['actu_dest_year'] = actu_dest.dt.year
    flight_final['actu_dest_month'] = actu_dest.dt.month
    flight_final['actu_dest_day'] = actu_dest.dt.day
    flight_final['actu_dest_hr'] = actu_dest.dt.hour
    flight_final['actu_dest_min'] = actu_dest.dt.minute

In [18]:
def save_data():
    flight_final.to_csv('data/flight_info_final_v2.csv', index= False)

In [19]:
if __name__ == "__main__":
    data_cleaning(),
    data_time_clearning(),
    save_data()